In [1]:
import numpy as np
import pandas as pd

In [2]:
def sigmoid(x):

    sigmoid = 1 / (1 + np.exp(-x))
    
    return sigmoid

def sigmoid_derivative(x):
    
    sigmoid_derivative = sigmoid(x) * (1-sigmoid(x))
    
    return sigmoid_derivative

In [3]:
class NeuralNetwork:
    def __init__(
            self,
            X_data,
            Y_data,
            n_hidden_neurons,
            n_categories,
            epochs,
            batch_size,
            eta,
            lmbd):

        self.X_data_full = X_data
        self.Y_data_full = Y_data

        self.n_inputs = X_data.shape[0]
        self.n_features = X_data.shape[1]
        self.n_hidden_neurons = n_hidden_neurons
        self.n_categories = n_categories

        self.epochs = epochs
        self.batch_size = batch_size
        self.iterations = self.n_inputs // self.batch_size
        self.eta = eta
        self.lmbd = lmbd

        self.create_biases_and_weights()

    def create_biases_and_weights(self):
        self.hidden_weights = np.random.randn(self.n_features, self.n_hidden_neurons)
        self.hidden_bias = np.zeros(self.n_hidden_neurons) + 0.01

        self.output_weights = np.random.randn(self.n_hidden_neurons, self.n_categories)
        self.output_bias = np.zeros(self.n_categories) + 0.01

    def feed_forward(self):
        # feed-forward for training
        self.z_h = np.matmul(self.X_data, self.hidden_weights) + self.hidden_bias
        self.a_h = sigmoid(self.z_h)

        self.z_o = np.matmul(self.a_h, self.output_weights) + self.output_bias
        self.a_o = sigmoid(self.z_o)
        #exp_term = np.exp(self.z_o)
        #print('exp_term SUM', np.sum(exp_term, keepdims=True))
        #self.probabilities = exp_term / np.sum(exp_term, keepdims=True)#, axis=1, keepdims=True)
        #print(self.probabilities)

    def feed_forward_out(self, X):
        # feed-forward for output
        z_h = np.matmul(X, self.hidden_weights) + self.hidden_bias
        a_h = sigmoid(z_h)

        z_o = np.matmul(a_h, self.output_weights) + self.output_bias
        a_o = sigmoid(z_o)
        #exp_term = np.exp(z_o)
        #probabilities = exp_term / np.sum(exp_term, keepdims=True)#, axis=1, keepdims=True)
        #return probabilities
        return a_o

    def backpropagation(self):
        #error_output = self.probabilities - self.Y_data
        error_output = self.a_o - self.Y_data
        error_hidden = np.matmul(error_output, self.output_weights.T) * self.a_h * (1 - self.a_h)

        self.output_weights_gradient = np.matmul(self.a_h.T, error_output)
        self.output_bias_gradient = np.sum(error_output, axis=0)

        self.hidden_weights_gradient = np.matmul(self.X_data.T, error_hidden)
        self.hidden_bias_gradient = np.sum(error_hidden, axis=0)

        if self.lmbd > 0.0:
            self.output_weights_gradient += self.lmbd * self.output_weights
            self.hidden_weights_gradient += self.lmbd * self.hidden_weights

        self.output_weights -= self.eta * self.output_weights_gradient
        self.output_bias -= self.eta * self.output_bias_gradient
        self.hidden_weights -= self.eta * self.hidden_weights_gradient
        self.hidden_bias -= self.eta * self.hidden_bias_gradient
        
        #print('here, have updated by', self.eta * self.output_weights_gradient)

    def predict(self, X):
        probabilities = self.feed_forward_out(X)
        #print(probabilities)
        #return np.argmax(probabilities, axis=1)
        return(probabilities)
        
    def predict_probabilities(self, X):
        probabilities = self.feed_forward_out(X)
        return probabilities

    def train(self):
        data_indices = np.arange(self.n_inputs)

        for i in range(self.epochs):
            for j in range(self.iterations):
                # pick datapoints with replacement
                chosen_datapoints = np.random.choice(
                    data_indices, size=self.batch_size, replace=True
                )

                # minibatch training data
                self.X_data = self.X_data_full[chosen_datapoints]
                self.Y_data = self.Y_data_full[chosen_datapoints]
                
                self.feed_forward()
                self.backpropagation()
                
                
    def printing(self):
        #print('prob', self.probabilities)
        print('O_w',  self.output_weights)
        print('O_b',  self.output_bias)
        print('H_w',  self.hidden_weights)
        print('H_B',  self.hidden_bias)

In [4]:
nanDict = {}
df = pd.read_excel('creditcard_data.xls', header=1, skiprows=0, index_col=0, na_values=nanDict)
df.rename(index=str, columns={"default payment next month": "defaultPaymentNextMonth"}, inplace=True)# Features and targets 

df = df.drop(df[(df.BILL_AMT1 == 0) &
                (df.BILL_AMT2 == 0) &
                (df.BILL_AMT3 == 0) &
                (df.BILL_AMT4 == 0) &
                (df.BILL_AMT5 == 0) &
                (df.BILL_AMT6 == 0)].index)

df = df.drop(df[(df.PAY_AMT1 == 0) &
                (df.PAY_AMT2 == 0) &
                (df.PAY_AMT3 == 0) &
                (df.PAY_AMT4 == 0) &
                (df.PAY_AMT5 == 0) &
                (df.PAY_AMT6 == 0)].index)

X = df.loc[:, df.columns != 'defaultPaymentNextMonth'].values
y = df.loc[:, df.columns == 'defaultPaymentNextMonth'].values

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

onehotencoder = OneHotEncoder(categories = "auto")

X = ColumnTransformer(
    [("", onehotencoder,  [1, 2, 3, 4]),],
    remainder="passthrough"
).fit_transform(X)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.5, test_size = 1-0.5)

In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler(with_mean=False)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# One-hot's of the target vector
y_train_onehot, y_test_onehot = onehotencoder.fit_transform(y_train), onehotencoder.fit_transform(y_test)

In [8]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [112]:
y_train_try = np.zeros(y_train.shape)

In [65]:
epochs = 5
n_inputs, n_features = X_train.shape
n_hidden_neurons = 50
n_categories = 1
epochs = 10
batch_size = 80
#iterations = n_inputs // batch_size
iterations = 10
eta = 0.001
lmbd = 0.0

In [116]:
dnn = NeuralNetwork(X_train.toarray(), y_train, eta=eta, lmbd=lmbd, epochs=epochs, batch_size=batch_size,
                    n_hidden_neurons=n_hidden_neurons, n_categories=n_categories)

In [117]:
dnn.train()
test_predict = dnn.predict(X_test.toarray())
test_predict[test_predict >= 0.5] = 1
test_predict[test_predict < 0.5] = 0

In [64]:
#dnn.printing()

In [118]:
# accuracy score from scikit library
print("Accuracy score on test set: ", accuracy_score(y_test, test_predict))
print(f1_score(y_test, test_predict))
print(confusion_matrix(y_test, test_predict))

Accuracy score on test set:  0.7992841602919503
0.2770475227502528
[[10841   374]
 [ 2486   548]]


In [120]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, f1_score

mlp = MLPClassifier(hidden_layer_sizes=(50,1), alpha=0.1, batch_size=80, learning_rate = 'constant', learning_rate_init = 10e-4, max_iter=100, activation='logistic')
mlp.fit(X_train.toarray(), np.ravel(y_train))
#print(mlp.score(X_test, y_test)

y_pred = mlp.predict(X_test.toarray())

print(accuracy_score(y_test, y_pred))
print(f1_score(y_pred, np.ravel(y_test)))
print(confusion_matrix(y_pred, np.ravel(y_test)))

0.7992841602919503
0.4802154547337892
[[10581  1875]
 [  634  1159]]
